In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8,8)
import geopandas as gpd
import seaborn
import matplotlib as mpl
from shapely.geometry import Point,LineString,LinearRing,Polygon
from shapely.geometry import MultiPoint,MultiLineString,MultiPolygon
%matplotlib inline
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# 整理数据代码

In [2]:
# 1 读取数据与处理
score = pd.read_csv('./data1/all.csv')

score

,index,wealthier,more depressing,safer,livelier,more boring,more beautiful
0,50e5f7d4d7c3df413b00056b,3.333333,3.333333,3.333333,4.444444,10.000000,0.666667
1,50e5f7d4d7c3df413b00056d,4.666667,3.333333,5.555556,3.333333,10.000000,3.333333
2,50e5f7d4d7c3df413b00056e,5.833333,4.444444,1.000000,3.611111,0.000000,0.000000
3,50e5f7d4d7c3df413b00056f,6.666667,5.000000,3.939394,5.555556,3.333333,4.444444
4,50e5f7d4d7c3df413b000570,5.000000,6.190476,2.666667,2.666667,10.000000,0.000000
...,...,...,...,...,...,...,...
96901,5185d51bfdc9f03fd500147a,5.000000,0.000000,6.666667,4.166667,2.500000,4.444444
96902,5185d52bfdc9f03fd5001486,0.000000,10.000000,1.111111,1.481481,10.000000,3.333333
96903,5185d531fdc9f03fd500148c,3.333333,6.666667,5.925926,2.857143,3.333333,5.714286
96904,5185d534fdc9f03fd500148f,10.000000,0.833333,6.190476,6.666667,5.000000,6.666667


In [3]:
# 1 读取数据与处理
score = pd.read_csv('./data1/all.csv')

perceptions = ['wealthier', 'more depressing', 'safer', 'livelier', 'more boring', 'more beautiful']
# 定义标签函数
def label(x,delta):
    if x < _mean - delta * _std:
        return -1
    elif x > _mean + delta * _std:
        return 1
    else:
        return 0
    
# 单个感知run
for perception in perceptions:
    score_s = score[['index',perception]].copy()

    _std, _mean = score_s[perception].describe()['std'],score_s[perception].describe()['mean']


    # 准备数据
    for delta in [0.5,0.7,1.0,1.2,1.5,1.8]:
        score_s[f'y_{perception}_{delta}'] = score_s[perception].apply(label,args=(delta,))

    # 3 读取语义分割的数据
    seg = pd.read_csv('./data1/seg_train.csv')
    seg = seg.fillna(0)
    seg['img'] = seg['img'].str.split('_').str.get(2)
    result = pd.merge(seg,score_s,left_on='img',right_on='index')

    # 4 训练#
    result_s = result[result[f'y_{perception}_1.2'] != 0]

    # 构建xy
    X = result_s.iloc[:,:150].values
    y = result_s[f'y_{perception}_1.2'].values
    
    unique_classes, counts = np.unique(y, return_counts=True)
    print("Classes found:", unique_classes)
    print("Class distribution:", counts)

    clf = SVC(kernel='rbf',C=0.001,probability=True)

    # 使用cross_val_predict获取交叉验证中的预测结果
    y_pred = cross_val_predict(clf, X, y, cv=5)

    # 计算指标，使用weighted平均处理多类问题
    print(perception,":",
          "Accuracy:", accuracy_score(y, y_pred),
          "Precision (weighted):", precision_score(y, y_pred, average='weighted'),
          "Recall (weighted):", recall_score(y, y_pred, average='weighted'),
          "F1 Score (weighted):", f1_score(y, y_pred, average='weighted'))

Classes found: [-1  1]
Class distribution: [16027 15953]
wealthier : Accuracy: 0.6236710444027517 Precision (weighted): 0.6295364991952925 Recall (weighted): 0.6236710444027517 F1 Score (weighted): 0.61912920924609
Classes found: [-1  1]
Class distribution: [16961 17137]
more depressing : Accuracy: 0.6189512581383072 Precision (weighted): 0.6278687715186579 Recall (weighted): 0.6189512581383072 F1 Score (weighted): 0.6115139776615387
Classes found: [-1  1]
Class distribution: [13005 12301]
safer : Accuracy: 0.6783371532442899 Precision (weighted): 0.6847328221715424 Recall (weighted): 0.6783371532442899 F1 Score (weighted): 0.6737428999228924
Classes found: [-1  1]
Class distribution: [13244 12781]
livelier : Accuracy: 0.677732949087416 Precision (weighted): 0.6784437862634546 Recall (weighted): 0.677732949087416 F1 Score (weighted): 0.6776794350377796
Classes found: [-1  1]
Class distribution: [16717 16378]
more boring : Accuracy: 0.5911769149418341 Precision (weighted): 0.59237257584

# 写循环，找精度最高的 1.0和1.8

In [9]:
# 1 读取数据与处理
score = pd.read_csv('./data1/all.csv')

score_s = score[['index','safer']].copy()

_std, _mean = score_s['safer'].describe()['std'],score_s['safer'].describe()['mean']


# 准备数据
for delta in [0.5,0.7,1.0,1.2,1.5,1.8]:
    score_s[f'y_safer_{delta}'] = score_s['safer'].apply(label,args=(delta,))

# 3 读取语义分割的数据
seg = pd.read_csv('./data1/seg_train.csv')
seg = seg.fillna(0)
seg['img'] = seg['img'].str.split('_').str.get(2)
result = pd.merge(seg,score_s,left_on='img',right_on='index')

In [11]:

number = [1.0,1.2,1.5,1.8]
for num in number:
    # 筛选不是0的
    result_s = result[result[f'y_safer_{num}'] != 0]
    
    # 构建xy
    X = result_s.iloc[:,:150].values
    y = result_s[f'y_safer_{num}'].values

    clf = SVC(kernel='rbf',C=0.001,probability=True)
    scores = cross_val_score(clf,X,y,cv=5)
    print(num)
    print(scores)
    print(np.mean(scores))
    print('-------------')


1.0
[0.67575569 0.67513244 0.68349696 0.67741935 0.67819854]
0.6780005956637949
-------------
1.2
[0.67996839 0.67634855 0.68148587 0.67417506 0.67970757]
0.6783370887811919
-------------
1.5
[0.65912829 0.66488487 0.65666118 0.65583882 0.64006582]
0.6553157948862284
-------------
1.8
[0.52891791 0.52941176 0.52941176 0.52847806 0.52847806]
0.5289395111278342
-------------
